<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

This notebook makes use of the HR hiring dataset, where protected attributes `gender` and `ethnicity` are not used to train the machine learning model that we are building as part of this notebook. However, you an configure OpenScale to monitor fairness for these attributes to check if the model is indirectly behaving in a biased manner.

This notebook should be run in a [Watson Studio project](https://dataplatform.cloud.ibm.com/projects/), using a Python 3.7 or above runtime environment. If you are viewing this in Watson Studio and do not see Python 3.7 or above in the upper right corner of your screen, please update the runtime now. It requires [Watson Machine Learning](https://cloud.ibm.com/catalog/services/machine-learning).

## Credentials for IBM Cloud Services

Your Cloud API key can be generated by going to the [Users section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it between the single quotes in the cell below.

In [ ]:
CLOUD_API_KEY = "___API_KEY_HERE___"

## Provide a project ID

To be able to save this model as a resource in your project, provide the project ID. It can be found as part of the URL for the current page as the __projectid__ identifier. Alternatively, if you insert a project token from the __More__ menu at the top of the screen (the three dots icon between the __Download__ and __Info__ buttons), you can find the project ID there. Note that doing this will insert a cell at the top of your notebook, so you may have to scroll up to find the key.

In [ ]:
project_id = '___PROJECT_ID_HERE___'

## Run the notebook

At this point, you should be able to run the remainder of the notebook without modifications. It will download training data from github and train a model. The model will be saved to your project as a model asset.

## Install packages and import libraries

In [ ]:
!pip install pyspark==2.4.0 --no-cache | tail -n 1

In [ ]:
import pandas as pd
from pyspark import SparkContext, SQLContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.sql.types import StructType, DoubleType, StringType, ArrayType

In [ ]:
WML_CREDENTIALS = {
    "apikey": CLOUD_API_KEY,
    "url": 'https://us-south.ml.cloud.ibm.com'
}
WML_CREDENTIALS

## Load and explore data

In [ ]:
!rm hr_hiring_data.csv
!wget https://raw.githubusercontent.com/ericmartens/indirect-bias/main/data/hr_training_data.csv

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles

pd_data = pd.read_csv("hr_training_data.csv", sep=",", header=0)

spark = SparkSession.builder.getOrCreate()
spark_df = spark.createDataFrame(pd_data)
spark_df.head()

## Create the model pipeline and train the model

Index the categorical fields from the training data

In [ ]:
si_BusinessTravel = StringIndexer(inputCol='BusinessTravel', outputCol='BusinessTravel_IX')
si_Department = StringIndexer(inputCol='Department', outputCol='Department_IX')
si_Education = StringIndexer(inputCol='Education', outputCol='Education_IX')
si_EducationField = StringIndexer(inputCol='EducationField', outputCol='EducationField_IX')
si_RelevantEducationLevel = StringIndexer(inputCol='RelevantEducationLevel', outputCol='RelevantEducationLevel_IX')
si_JobRole = StringIndexer(inputCol='JobRole', outputCol='JobRole_IX')
si_JobLevel = StringIndexer(inputCol='JobLevel', outputCol='JobLevel_IX')
si_MaritalStatus = StringIndexer(inputCol='MaritalStatus', outputCol='MaritalStatus_IX')
si_OverTime = StringIndexer(inputCol='OverTime', outputCol='OverTime_IX')
si_RequestedBenefits = StringIndexer(inputCol='RequestedBenefits', outputCol='RequestedBenefits_IX')
si_PreferredSkills = StringIndexer(inputCol='PreferredSkills', outputCol='PreferredSkills_IX')
si_JobType = StringIndexer(inputCol='JobType', outputCol='JobType_IX')
si_SalaryExpectation = StringIndexer(inputCol='SalaryExpectation', outputCol='SalaryExpectation_IX')

si_InterviewScore = StringIndexer(inputCol='InterviewScore', outputCol='InterviewScore_IX')
si_ResumeScore = StringIndexer(inputCol='ResumeScore', outputCol='ResumeScore_IX')

si_Label = StringIndexer(inputCol="HIRED", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

Drop the gender and ethnicity columns, since we will not be using these to train the model. OpenScale will be configured to monitor these values.

In [ ]:
columns_to_drop = ['Gender', 'Ethnicity']
spark_df_tmp = spark_df.drop(*columns_to_drop)

Perform the train/test split.

In [ ]:
(train_data, test_data) = spark_df_tmp.randomSplit([0.9, 0.1], 24)
print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

In [ ]:
va_features = VectorAssembler(
inputCols=["Age", "BusinessTravel_IX", "Department_IX", "DistanceFromHome",
           "Education_IX", "EducationField_IX", "RelevantEducationLevel_IX", "JobLevel_IX", "JobRole_IX"
           , "MaritalStatus_IX","NumCompaniesWorked", "OverTime_IX",
           "InterviewScore_IX", "ResumeScore_IX", "RequestedBenefits_IX", "TotalWorkingYears", "PreferredSkills_IX",
           "YearsAtCurrentCompany","RelevantExperience","JobType_IX","SalaryExpectation_IX"], outputCol="features")

In [ ]:
classifier = GBTClassifier(featuresCol="features")

Construct the model pipeline

In [ ]:
pipeline = Pipeline(stages=[si_BusinessTravel, si_Department,si_Education, si_EducationField,si_RelevantEducationLevel, si_JobRole,si_JobLevel, si_MaritalStatus,
        si_OverTime,si_InterviewScore,si_ResumeScore,si_RequestedBenefits,si_PreferredSkills,si_JobType,si_SalaryExpectation, si_Label, va_features, classifier, label_converter])

Evaluate the model

In [ ]:
model = pipeline.fit(train_data)
predictions = model.transform(test_data)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = %g" % accuracy)

# Save the model

Save the model as a model asset to your Watson Studio project.

In [ ]:
wml_client.set.default_project(project_id)

In [ ]:
model_name = 'PySpark Hiring Challenger'
software_spec_uid = wml_client.software_specifications.get_uid_by_name('spark-mllib_2.4')
model_props = {
    wml_client.repository.ModelMetaNames.NAME: model_name,
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client.repository.ModelMetaNames.TYPE: "mllib_2.4"
}
stored_model_details = wml_client.repository.store_model(model=model, pipeline=pipeline, meta_props=model_props, training_data=train_data)

## Congratulations

At this point, the model will be available in your project. You can promote it to a deployment space and then deploy it as a web service.